In [301]:
import numpy as np
import scipy as sp
import scipy.special as scisp
import scipy.integrate as integ
import mpmath
from scipy import constants as const
import Green_functions as gr
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
sim = gr.simulation()
import scipy as sc
import nanonis
import scipy.signal as sps
import deconvolution
%matplotlib qt

In [3]:
# DEFINITIONS

In [268]:
def line(t,a,b,g):
    return a+1j*b+g*np.exp(1j*t)

#def res(w,gamma,delta,vF,mu,i,j):
#    y=[]
#    for l in gamma:
#        y.append(G0(w,l,delta,vF,mu)[i,j])
#    return np.array(y)

def res(x,w,delta,vF,mu,pole,g):
    t=np.linspace(0,2*np.pi,1000)
    gamma=line(t,np.real(pole),np.imag(pole),g)
    return np.trapz(g*G0(w,gamma,delta,vF,mu)*np.exp(1j*t),t)*np.exp(1j*pole*x)

def res0(x,w,delta,vF,mu,pole,g):
    t=np.linspace(0,2*np.pi,1000)
    gamma=line(t,np.real(pole),np.imag(pole),g)
    return np.trapz(g*(G0(w,gamma,delta,vF,mu)+G0(w,-gamma,delta,vF,mu))*np.exp(1j*t),t)*np.exp(1j*pole*x)/2

def H0(p,delta,vF,mu):
    return [[vF*p -mu, 0, delta, 0], [0, -vF*p - mu, 0, delta], [delta, 0, -vF*p + mu, 0], [0,delta, 0, vF*p + mu]]

def HYSR(Jx,Jy,Jz):
   return np.kron(np.identity(2),[[Jz,Jx-1j*Jy],[Jx+1J*Jy,-Jz]])

#def G0(w,p,delta,vF,mu):
#   return np.linalg.inv(w*np.identity(4)-H0(p,delta,vF,mu))

def G0(w,p,delta,vF,mu):
    a=np.array([[(p*vF - mu +w)*(-p**2*vF**2 - delta**2 - 2*p*vF*mu - mu**2 +w**2), 0, -p**2*vF**2*delta - delta**3 - 2*p*vF*delta*mu - delta*mu**2 + delta*w**2, 0],\
    [0, (-p*vF - mu +w)*(-p**2*vF**2 - delta**2 + 2*p*vF*mu - mu**2 +w**2), 0, -p**2*vF**2*delta - delta**3 + 2*p*vF*delta*mu - delta*mu**2 + delta*w**2],\
    [-p**2*vF**2*delta - delta**3 - 2*p*vF*delta*mu - delta*mu**2 + delta*w**2,  0, (-p*vF + mu +w)*(-p**2*vF**2 - delta**2 - 2*p*vF*mu - mu**2 +w**2), 0],\
    [0, -p**2*vF**2*delta - delta**3 + 2*p*vF*delta*mu - delta*mu**2 + delta*w**2, 0, (p*vF + mu +w)*(-p**2*vF**2 - delta**2 + 2*p*vF*mu - mu**2 +w**2)]])
    det=(vF**4*p**4+2*vF**2*(delta**2-mu**2-w**2)*p**2+(delta**2+mu**2-w**2)**2)
    return a
def det(w,delta,vF,mu):
    coef=[vF**4,0,2*vF**2*(delta**2-mu**2-w**2),0,(delta**2+mu**2-w**2)**2]
    return np.roots(coef)

def sort(x):
    p=[]
    m=[]
    for i in range(len(x)):
        if np.imag(x[i])>0:
            p.append(x[i])
        else:
            m.append(x[i])
    return p,m

#def G(x,w,delta,vF,mu,g,k,l):
#    poles=det(w,delta,vF,mu)
#    pp,pm=sort(poles)
#    t=np.linspace(0,2*np.pi,1000)
#
#    if x>0.001:
#        temp=0
#        for i in pp:
#            gamma=line(t,np.real(i),np.imag(i),g)
#            temp+=np.trapz(g*res(w,gamma,delta,vF,mu,k,l)*np.exp(1j*t),t)*np.exp(1j*i*x)/(2*np.pi)
#        return temp
#
#    if x<-0.001:
#        temp=0
#        for i in pm:
#            a=np.real(i)
#            b=np.imag(i)
#            gamma=line(t,a,b,g)
#            temp+=np.trapz(g*res(w,gamma,delta,vF,mu,k,l)*np.exp(1j*t),t)*np.exp(1j*i*x)/(-2*np.pi)
#        return temp
#    else:
#        temp=0
#        for i in pm:
#            a=np.real(i)
#            b=np.imag(i)
#            gamma=line(t,a,b,g)
#            temp+=np.trapz(g*(res(w,gamma,delta,vF,mu,k,l)+res(w,-gamma,delta,vF,mu,k,l))*1j*np.exp(1j*t),t)*np.exp(1j*i*x)/(4*np.pi*1j)
#        return temp

def G(x,w,delta,vF,mu,g):
    poles=det(w,delta,vF,mu)
    pp,pm=sort(poles)
    if x>0.001:
        temp=0
        for i in pp:
            temp+=res(x,w,delta,vF,mu,i,g)
        return temp

    if x<-0.001:
        temp=0
        for i in pm:
            temp-=res(x,w,delta,vF,mu,i,g)
        return temp
    else:
        temp=0
        for i in pp:
            temp+=res0(x,w,delta,vF,mu,i,g)
        return temp
    
def G0real(x,w,delta,vF,mu):
    G=np.zeros([4,4])+0*1j
    for i in range(4):
        for j in range(4):
            G[i,j]=resG(x,w,delta,vF,mu,i,j)
    return G

def GYSR(x,w,delta,vF,mu,Jx,Jy,Jz):
    g0=G0real(0,w,delta,vF,mu)
    T=np.dot(HYSR(Jx,Jy,Jz),np.linalg.inv(np.identity(4)-np.dot(g0,HYSR(Jx,Jy,Jz))))
    return g0+np.dot(g0,np.dot(T,g0))

#w=np.linspace(-2,2,100)
#y=[]
#for i in w:
#    print(i)
#    y.append(-np.real(G(0,i+0.001*1j,1,2,100,0.001)))
#
#plt.plot(w,y)
G0(2+0.001*1j,np.array([1,2]),1,2,100)/np.array([1,2])



C:\Users\jonor\AppData\Local\Temp/ipykernel_17336/3208680664.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a=np.array([[(p*vF - mu +w)*(-p**2*vF**2 - delta**2 - 2*p*vF*mu - mu**2 +w**2), 0, -p**2*vF**2*delta - delta**3 - 2*p*vF*delta*mu - delta*mu**2 + delta*w**2, 0],\


ValueError: operands could not be broadcast together with shapes (4,4) (2,) 

In [190]:
def fproba(x):
    return np.array([1/(x**2+4),1/(x**2+4)])

def line(t,a,b,g):
    return a+1j*b+g*np.exp(1j*t)

def residue(a,b,g):
    t=np.linspace(0,2*np.pi,1000)
    x=line(t,a,b,g)
    y=g*fproba(x)*1j*np.exp(1j*t)
    return np.trapz(y,t)

residue(0,2,0.00001)/(2*np.pi)


array([0.25+1.6428532e-14j, 0.25+1.6428532e-14j])

In [342]:
A=[[0., 640.003], [0.01, 786.499], [0.02, 771.64], [0.03, 755.521], [0.04, 738.25], [0.05, 719.947], [0.06, 700.744], [0.07, 680.78], [0.08, 660.204], [0.09, 639.17], [0.1, 617.835], [0.11, 596.358], [0.12, 574.898], [0.13, 553.612], [0.14, 532.652], [0.15, 512.167], [0.16, 492.298], [0.17, 473.177], [0.18, 454.926], [0.19, 437.657], [0.2, 421.467], [0.21, 406.444], [0.22, 392.659], [0.23, 380.17], [0.24, 369.018], [0.25, 359.232], [0.26, 350.825], [0.27, 343.792], [0.28, 338.118], [0.29, 333.77], [0.3, 330.703], [0.31, 328.859], [0.32, 328.169], [0.33, 328.551], [0.34, 329.914], [0.35, 332.159], [0.36, 335.18], [0.37, 338.863], [0.38, 343.092], [0.39, 347.746], [0.4, 352.703], [0.41, 357.84], [0.42, 363.037], [0.43, 368.174], [0.44, 373.136], [0.45, 377.814], [0.46, 382.103], [0.47, 385.906], [0.48, 389.136], [0.49, 391.712], [0.5, 393.564], [0.51, 394.634], [0.52, 394.873], [0.53, 394.243], [0.54, 392.719], [0.55, 390.285], [0.56, 386.939], [0.57, 382.689], [0.58, 377.554], [0.59, 371.563], [0.6, 364.755], [0.61, 357.179], [0.62, 348.892], [0.63, 339.959], [0.64, 330.451], [0.65, 320.446], [0.66, 310.027], [0.67, 299.278], [0.68, 288.289], [0.69, 277.15], [0.7, 265.953], [0.71, 254.787], [0.72, 243.742], [0.73, 232.903], [0.74, 222.354], [0.75, 212.174], [0.76, 202.435], [0.77, 193.205], [0.78, 184.544], [0.79, 176.507], [0.8, 169.139], [0.81, 162.478], [0.82, 156.553], [0.83, 151.385], [0.84, 146.987], [0.85, 143.362], [0.86, 140.506], [0.87, 138.405], [0.88, 137.038], [0.89, 136.376], [0.9, 136.385], [0.91, 137.02], [0.92, 138.235], [0.93, 139.975], [0.94, 142.182], [0.95, 144.794], [0.96, 147.745], [0.97, 150.969], [0.98, 154.396], [0.99, 157.957], [1., 161.581], [1.01, 165.202], [1.02, 168.751], [1.03, 172.165], [1.04, 175.382], [1.05, 178.346], [1.06, 181.002], [1.07, 183.302], [1.08, 185.204], [1.09, 186.67], [1.1, 187.67], [1.11, 188.178], [1.12, 188.175], [1.13, 187.65], [1.14, 186.598], [1.15, 185.019], [1.16, 182.92], [1.17, 180.315], [1.18, 177.224], [1.19, 173.67], [1.2, 169.684], [1.21, 165.301], [1.22, 160.558], [1.23, 155.499], [1.24, 150.168], [1.25, 144.614], [1.26, 138.887], [1.27, 133.037], [1.28, 127.117], [1.29, 121.178], [1.3, 115.271], [1.31, 109.448], [1.32, 103.756], [1.33, 98.241], [1.34, 92.948], [1.35, 87.9168], [1.36, 83.1845], [1.37, 78.7842], [1.38, 74.7445], [1.39, 71.0899], [1.4, 67.8401], [1.41, 65.0098], [1.42, 62.6091], [1.43, 60.643], [1.44, 59.1116], [1.45, 58.0104], [1.46, 57.33], [1.47, 57.0567], [1.48, 57.1724], [1.49, 57.6554], [1.5, 58.4799], [1.51, 59.6173], [1.52, 61.0359], [1.53, 62.7016], [1.54, 64.5783], [1.55, 66.6282], [1.56, 68.8127], [1.57, 71.0921], [1.58, 73.4271], [1.59, 75.7781], [1.6, 78.1066], [1.61, 80.3751], [1.62, 82.5477], [1.63, 84.5904], [1.64, 86.4715], [1.65, 88.1619], [1.66, 89.6355], [1.67, 90.869], [1.68, 91.8428], [1.69, 92.5407], [1.7, 92.9498], [1.71, 93.0614], [1.72, 92.8701], [1.73, 92.3745], [1.74, 91.5766], [1.75, 90.482], [1.76, 89.0999], [1.77, 87.4427], [1.78, 85.5256], [1.79, 83.3669], [1.8, 80.9873], [1.81, 78.4097], [1.82, 75.6592], [1.83, 72.7623], [1.84, 69.7467], [1.85, 66.6414], [1.86, 63.4757], [1.87, 60.2792], [1.88, 57.0814], [1.89, 53.9113], [1.9, 50.7974], [1.91, 47.7667], [1.92, 44.8453], [1.93, 42.0571], [1.94, 39.4247], [1.95, 36.9681], [1.96, 34.7052], [1.97, 32.6516], [1.98, 30.8199], [1.99, 29.2203], [2., 27.8601], [2.01, 26.744], [2.02, 25.8736], [2.03, 25.2479], [2.04, 24.8633], [2.05, 24.7133], [2.06, 24.7892], [2.07, 25.0798], [2.08, 25.5718], [2.09, 26.2499], [2.1, 27.0969], [2.11, 28.0943], [2.12, 29.2221], [2.13, 30.4594], [2.14, 31.7845], [2.15, 33.1751], [2.16, 34.6086], [2.17, 36.0627], [2.18, 37.5149], [2.19, 38.9436], [2.2, 40.3278], [2.21, 41.6475], [2.22, 42.8839], [2.23, 44.0194], [2.24, 45.0383], [2.25, 45.9263], [2.26, 46.6711], [2.27, 47.2622], [2.28, 47.6913], [2.29, 47.952], [2.3, 48.0401], [2.31, 47.9536], [2.32, 47.6924], [2.33, 47.2586], [2.34, 46.6563], [2.35, 45.8915], [2.36, 44.9719], [2.37, 43.907], [2.38, 42.7078], [2.39, 41.3867], [2.4, 39.9572], [2.41, 38.4341], [2.42, 36.8328], [2.43, 35.1694], [2.44, 33.4605], [2.45, 31.7229], [2.46, 29.9736], [2.47, 28.2293], [2.48, 26.5064], [2.49, 24.8208], [2.5, 23.1878], [2.51, 21.6217], [2.52, 20.1359], [2.53, 18.7426], [2.54, 17.4529], [2.55, 16.2764], [2.56, 15.2212], [2.57, 14.2941], [2.58, 13.5002], [2.59, 12.843], [2.6, 12.3245], [2.61, 11.945], [2.62, 11.7034], [2.63, 11.5968], [2.64, 11.6212], [2.65, 11.7709], [2.66, 12.0391], [2.67, 12.4177], [2.68, 12.8975], [2.69, 13.4685], [2.7, 14.1197], [2.71, 14.8395], [2.72, 15.6159], [2.73, 16.4363], [2.74, 17.2879], [2.75, 18.1579], [2.76, 19.0335], [2.77, 19.9022], [2.78, 20.7518], [2.79, 21.5705], [2.8, 22.3472], [2.81, 23.0714], [2.82, 23.7337], [2.83, 24.3254], [2.84, 24.8387], [2.85, 25.2672], [2.86, 25.6054], [2.87, 25.8489], [2.88, 25.9948], [2.89, 26.041], [2.9, 25.987], [2.91, 25.8331], [2.92, 25.5812], [2.93, 25.2339], [2.94, 24.7952], [2.95, 24.2698], [2.96, 23.6635], [2.97, 22.983], [2.98, 22.2356], [2.99, 21.4293], [3., 20.5727], [3.01, 19.6747], [3.02, 18.7448], [3.03, 17.7924], [3.04, 16.8272], [3.05, 15.8587], [3.06, 14.8965], [3.07, 13.9499], [3.08, 13.0276], [3.09, 12.1383], [3.1, 11.2899], [3.11, 10.4898], [3.12, 9.7447], [3.13, 9.0605], [3.14, 8.44239], [3.15, 7.89471], [3.16, 7.42089], [3.17, 7.02351], [3.18, 6.70421], [3.19, 6.46374], [3.2, 6.30191], [3.21, 6.21771], [3.22, 6.20922], [3.23, 6.27373], [3.24, 6.40776], [3.25, 6.60711], [3.26, 6.86693], [3.27, 7.18177], [3.28, 7.54569], [3.29, 7.9523], [3.3, 8.39487], [3.31, 8.86637], [3.32, 9.35963], [3.33, 9.86734], [3.34, 10.3822], [3.35, 10.897], [3.36, 11.4046], [3.37, 11.8982], [3.38, 12.3713], [3.39, 12.8176], [3.4, 13.2314], [3.41, 13.6076], [3.42, 13.9413], [3.43, 14.2286], [3.44, 14.4658], [3.45, 14.6502], [3.46, 14.7796], [3.47, 14.8526], [3.48, 14.8683], [3.49, 14.8266], [3.5, 14.7282], [3.51, 14.5741], [3.52, 14.3664], [3.53, 14.1074], [3.54, 13.8], [3.55, 13.4479], [3.56, 13.0549], [3.57, 12.6255], [3.58, 12.1642], [3.59, 11.6762], [3.6, 11.1666], [3.61, 10.6407], [3.62, 10.1042], [3.63, 9.56238], [3.64, 9.02078], [3.65, 8.48477], [3.66, 7.95953], [3.67, 7.45007], [3.68, 6.96111], [3.69, 6.49707], [3.7, 6.06198], [3.71, 5.6595], [3.72, 5.29283], [3.73, 4.96475], [3.74, 4.67749], [3.75, 4.43283], [3.76, 4.23201], [3.77, 4.07576], [3.78, 3.96429], [3.79, 3.8973], [3.8, 3.87399], [3.81, 3.8931], [3.82, 3.95289], [3.83, 4.0512], [3.84, 4.18548], [3.85, 4.35284], [3.86, 4.55005], [3.87, 4.77361], [3.88, 5.01979], [3.89, 5.28469], [3.9, 5.56427], [3.91, 5.8544], [3.92, 6.15092], [3.93, 6.44967], [3.94, 6.74658], [3.95, 7.03766], [3.96, 7.31907], [3.97, 7.58716], [3.98, 7.83852], [3.99, 8.07], [4., 8.27874]]
B=[[-1.5, 1.95638], [-1.495, 1.95583], [-1.49, 1.95526], [-1.485,   1.95468], [-1.48, 1.9541], [-1.475, 1.9535], [-1.47,   1.9529], [-1.465, 1.95228], [-1.46, 1.95166], [-1.455,   1.95102], [-1.45, 1.95037], [-1.445, 1.94971], [-1.44,   1.94904], [-1.435, 1.94836], [-1.43, 1.94767], [-1.425,   1.94697], [-1.42, 1.94625], [-1.415, 1.94552], [-1.41,   1.94478], [-1.405, 1.94403], [-1.4, 1.94326], [-1.395,   1.94248], [-1.39, 1.94169], [-1.385, 1.94089], [-1.38,   1.94007], [-1.375, 1.93924], [-1.37, 1.93839], [-1.365,   1.93753], [-1.36, 1.93666], [-1.355, 1.93577], [-1.35,   1.93487], [-1.345, 1.93396], [-1.34, 1.93303], [-1.335,   1.93208], [-1.33, 1.93113], [-1.325, 1.93015], [-1.32,   1.92917], [-1.315, 1.92817], [-1.31, 1.92716], [-1.305,   1.92613], [-1.3, 1.92509], [-1.295, 1.92403], [-1.29,   1.92297], [-1.285, 1.92189], [-1.28, 1.9208], [-1.275,   1.9197], [-1.27, 1.91859], [-1.265, 1.91747], [-1.26,   1.91634], [-1.255, 1.91521], [-1.25, 1.91407], [-1.245,   1.91292], [-1.24, 1.91178], [-1.235, 1.91063], [-1.23,   1.90949], [-1.225, 1.90835], [-1.22, 1.90722], [-1.215,   1.90611], [-1.21, 1.90501], [-1.205, 1.90392], [-1.2,   1.90287], [-1.195, 1.90185], [-1.19, 1.90087], [-1.185,   1.89993], [-1.18, 1.89905], [-1.175, 1.89824], [-1.17,   1.8975], [-1.165, 1.89686], [-1.16, 1.89632], [-1.155,   1.89591], [-1.15, 1.89564], [-1.145, 1.89555], [-1.14,   1.89565], [-1.135, 1.89598], [-1.13, 1.89658], [-1.125,   1.8975], [-1.12, 1.89878], [-1.115, 1.90048], [-1.11,   1.9027], [-1.105, 1.9055], [-1.1, 1.909], [-1.095, 1.91334], [-1.09,   1.91866], [-1.085, 1.92517], [-1.08, 1.93311], [-1.075,   1.94279], [-1.07, 1.9546], [-1.065, 1.96904], [-1.06,   1.9868], [-1.055, 2.00874], [-1.05, 2.03611], [-1.045,   2.07062], [-1.04, 2.11475], [-1.035, 2.17227], [-1.03,   2.24917], [-1.025, 2.35558], [-1.02, 2.51029], [-1.015,   2.75299], [-1.01, 3.18664], [-1.005, 4.2057], [-1.,   6.4085], [-0.995, 0.417398], [-0.99, 0.160302], [-0.985,   0.0933618], [-0.98, 0.064705], [-0.975, 0.0493457], [-0.97,   0.0399801], [-0.965, 0.0337721], [-0.96, 0.0294117], [-0.955,   0.0262174], [-0.95, 0.0238027], [-0.945, 0.0219336], [-0.94,   0.0204606], [-0.935, 0.0192844], [-0.93, 0.0183366], [-0.925,   0.0175687], [-0.92, 0.0169455], [-0.915, 0.016441], [-0.91,   0.0160357], [-0.905, 0.0157146], [-0.9, 0.0154661], [-0.895,   0.0152814], [-0.89, 0.0151536], [-0.885, 0.0150771], [-0.88,   0.0150478], [-0.875, 0.0150625], [-0.87, 0.0151187], [-0.865,   0.0152147], [-0.86, 0.0153495], [-0.855, 0.0155223], [-0.85,   0.015733], [-0.845, 0.0159819], [-0.84, 0.0162696], [-0.835,   0.0165974], [-0.83, 0.0169666], [-0.825, 0.0173791], [-0.82,   0.0178374], [-0.815, 0.0183441], [-0.81, 0.0189027], [-0.805,   0.019517], [-0.8, 0.0201916], [-0.795, 0.0209315], [-0.79,   0.0217427], [-0.785, 0.0226323], [-0.78, 0.0236082], [-0.775,   0.0246796], [-0.77, 0.0258571], [-0.765, 0.0271531], [-0.76,   0.028582], [-0.755, 0.0301605], [-0.75, 0.0319085], [-0.745,   0.033849], [-0.74, 0.0360095], [-0.735, 0.0384228], [-0.73,   0.0411278], [-0.725, 0.0441718], [-0.72, 0.0476119], [-0.715,   0.0515181], [-0.71, 0.0559764], [-0.705, 0.0610942], [-0.7,   0.0670061], [-0.695, 0.0738832], [-0.69, 0.0819453], [-0.685,   0.0914789], [-0.68, 0.102863], [-0.675, 0.116605], [-0.67,   0.133404], [-0.665, 0.154234], [-0.66, 0.180492], [-0.655,   0.214238], [-0.65, 0.25861], [-0.645, 0.318582], [-0.64,   0.402415], [-0.635, 0.524665], [-0.63, 0.712943], [-0.625,   1.02496], [-0.62, 1.59859], [-0.615, 2.83442], [-0.61,   6.33918], [-0.605, 24.6179], [-0.6, 640.003], [-0.595,   24.6179], [-0.59, 6.33911], [-0.585, 2.83432], [-0.58,   1.59845], [-0.575, 1.02479], [-0.57, 0.712738], [-0.565,   0.524425], [-0.56, 0.40214], [-0.555, 0.318271], [-0.55,   0.258263], [-0.545, 0.213855], [-0.54, 0.180072], [-0.535,   0.153777], [-0.53, 0.132909], [-0.525, 0.116071], [-0.52,   0.102289], [-0.515, 0.0908647], [-0.51, 0.0812898], [-0.505,   0.0731855], [-0.5, 0.0662651], [-0.495, 0.0603089], [-0.49,   0.0551455], [-0.485, 0.0506402], [-0.48, 0.0466858], [-0.475,   0.0431959], [-0.47, 0.0401005], [-0.465, 0.0373424], [-0.46,   0.0348741], [-0.455, 0.0326566], [-0.45, 0.030657], [-0.445,   0.0288476], [-0.44, 0.027205], [-0.435, 0.0257095], [-0.43,   0.024344], [-0.425, 0.0230939], [-0.42, 0.0219465], [-0.415,   0.0208909], [-0.41, 0.0199177], [-0.405, 0.0190185], [-0.4,   0.0181861], [-0.395, 0.017414], [-0.39, 0.0166965], [-0.385,   0.0160287], [-0.38, 0.0154062], [-0.375, 0.0148249], [-0.37,   0.0142814], [-0.365, 0.0137724], [-0.36, 0.0132952], [-0.355,   0.0128473], [-0.35, 0.0124262], [-0.345, 0.01203], [-0.34,   0.0116568], [-0.335, 0.0113049], [-0.33, 0.0109726], [-0.325,   0.0106588], [-0.32, 0.0103619], [-0.315, 0.010081], [-0.31,   0.00981487], [-0.305, 0.00956258], [-0.3, 0.00932324], [-0.295,   0.00909601], [-0.29, 0.00888015], [-0.285, 0.00867495], [-0.28,   0.00847977], [-0.275, 0.00829402], [-0.27, 0.00811714], [-0.265,   0.00794864], [-0.26, 0.00778803], [-0.255, 0.00763489], [-0.25,   0.0074888], [-0.245, 0.0073494], [-0.24, 0.00721632], [-0.235,   0.00708926], [-0.23, 0.00696789], [-0.225, 0.00685195], [-0.22,   0.00674115], [-0.215, 0.00663527], [-0.21, 0.00653406], [-0.205,   0.00643732], [-0.2, 0.00634483], [-0.195, 0.00625642], [-0.19,   0.0061719], [-0.185, 0.00609111], [-0.18, 0.00601389], [-0.175,   0.00594009], [-0.17, 0.00586959], [-0.165, 0.00580225], [-0.16,   0.00573795], [-0.155, 0.00567658], [-0.15, 0.00561804], [-0.145,   0.00556221], [-0.14, 0.00550901], [-0.135, 0.00545835], [-0.13,   0.00541015], [-0.125, 0.00536433], [-0.12, 0.00532082], [-0.115,   0.00527955], [-0.11, 0.00524045], [-0.105, 0.00520347], [-0.1,   0.00516855], [-0.095, 0.00513563], [-0.09, 0.00510468], [-0.085,   0.00507563], [-0.08, 0.00504846], [-0.075, 0.00502312], [-0.07,   0.00499958], [-0.065, 0.0049778], [-0.06, 0.00495775], [-0.055,   0.0049394], [-0.05, 0.00492273], [-0.045, 0.00490772], [-0.04,   0.00489434], [-0.035, 0.00488258], [-0.03, 0.00487242], [-0.025,   0.00486384], [-0.02, 0.00485684], [-0.015, 0.00485141], [-0.01,   0.00484753], [-0.005, 0.00484521], [0., 0.00484443], [0.005,   0.00484521], [0.01, 0.00484753], [0.015, 0.00485141], [0.02,   0.00485684], [0.025, 0.00486384], [0.03, 0.00487242], [0.035,   0.00488258], [0.04, 0.00489434], [0.045, 0.00490772], [0.05,   0.00492273], [0.055, 0.0049394], [0.06, 0.00495775], [0.065,   0.0049778], [0.07, 0.00499958], [0.075, 0.00502312], [0.08,   0.00504846], [0.085, 0.00507563], [0.09, 0.00510468], [0.095,   0.00513563], [0.1, 0.00516855], [0.105, 0.00520347], [0.11,   0.00524045], [0.115, 0.00527955], [0.12, 0.00532082], [0.125,   0.00536433], [0.13, 0.00541015], [0.135, 0.00545835], [0.14,   0.00550901], [0.145, 0.00556221], [0.15, 0.00561804], [0.155,   0.00567658], [0.16, 0.00573795], [0.165, 0.00580225], [0.17,   0.00586959], [0.175, 0.00594009], [0.18, 0.00601389], [0.185,   0.00609111], [0.19, 0.0061719], [0.195, 0.00625642], [0.2,   0.00634483], [0.205, 0.00643732], [0.21, 0.00653406], [0.215,   0.00663527], [0.22, 0.00674115], [0.225, 0.00685195], [0.23,   0.00696789], [0.235, 0.00708926], [0.24, 0.00721632], [0.245,   0.0073494], [0.25, 0.0074888], [0.255, 0.00763489], [0.26,   0.00778803], [0.265, 0.00794864], [0.27, 0.00811714], [0.275,   0.00829402], [0.28, 0.00847977], [0.285, 0.00867495], [0.29,   0.00888015], [0.295, 0.00909601], [0.3, 0.00932324], [0.305,   0.00956258], [0.31, 0.00981487], [0.315, 0.010081], [0.32,   0.0103619], [0.325, 0.0106588], [0.33, 0.0109726], [0.335,   0.0113049], [0.34, 0.0116568], [0.345, 0.01203], [0.35,   0.0124262], [0.355, 0.0128473], [0.36, 0.0132952], [0.365,   0.0137724], [0.37, 0.0142814], [0.375, 0.0148249], [0.38,   0.0154062], [0.385, 0.0160287], [0.39, 0.0166965], [0.395,   0.017414], [0.4, 0.0181861], [0.405, 0.0190185], [0.41,   0.0199177], [0.415, 0.0208909], [0.42, 0.0219465], [0.425,   0.0230939], [0.43, 0.024344], [0.435, 0.0257095], [0.44,   0.027205], [0.445, 0.0288476], [0.45, 0.030657], [0.455,   0.0326566], [0.46, 0.0348741], [0.465, 0.0373424], [0.47,   0.0401005], [0.475, 0.0431959], [0.48, 0.0466858], [0.485,   0.0506402], [0.49, 0.0551455], [0.495, 0.0603089], [0.5,   0.0662651], [0.505, 0.0731855], [0.51, 0.0812898], [0.515,   0.0908647], [0.52, 0.102289], [0.525, 0.116071], [0.53,   0.132909], [0.535, 0.153777], [0.54, 0.180072], [0.545,   0.213855], [0.55, 0.258263], [0.555, 0.318271], [0.56,   0.40214], [0.565, 0.524425], [0.57, 0.712738], [0.575,   1.02479], [0.58, 1.59845], [0.585, 2.83432], [0.59,   6.33911], [0.595, 24.6179], [0.6, 640.003], [0.605, 24.6179], [0.61,   6.33918], [0.615, 2.83442], [0.62, 1.59859], [0.625,   1.02496], [0.63, 0.712943], [0.635, 0.524665], [0.64,   0.402415], [0.645, 0.318582], [0.65, 0.25861], [0.655,   0.214238], [0.66, 0.180492], [0.665, 0.154234], [0.67,   0.133404], [0.675, 0.116605], [0.68, 0.102863], [0.685,   0.0914789], [0.69, 0.0819453], [0.695, 0.0738832], [0.7,   0.0670061], [0.705, 0.0610942], [0.71, 0.0559764], [0.715,   0.0515181], [0.72, 0.0476119], [0.725, 0.0441718], [0.73,   0.0411278], [0.735, 0.0384228], [0.74, 0.0360095], [0.745,   0.033849], [0.75, 0.0319085], [0.755, 0.0301605], [0.76,   0.028582], [0.765, 0.0271531], [0.77, 0.0258571], [0.775,   0.0246796], [0.78, 0.0236082], [0.785, 0.0226323], [0.79,   0.0217427], [0.795, 0.0209315], [0.8, 0.0201916], [0.805,   0.019517], [0.81, 0.0189027], [0.815, 0.0183441], [0.82,   0.0178374], [0.825, 0.0173791], [0.83, 0.0169666], [0.835,   0.0165974], [0.84, 0.0162696], [0.845, 0.0159819], [0.85,   0.015733], [0.855, 0.0155223], [0.86, 0.0153495], [0.865,   0.0152147], [0.87, 0.0151187], [0.875, 0.0150625], [0.88,   0.0150478], [0.885, 0.0150771], [0.89, 0.0151536], [0.895,   0.0152814], [0.9, 0.0154661], [0.905, 0.0157146], [0.91,   0.0160357], [0.915, 0.016441], [0.92, 0.0169455], [0.925,   0.0175687], [0.93, 0.0183366], [0.935, 0.0192844], [0.94,   0.0204606], [0.945, 0.0219336], [0.95, 0.0238027], [0.955,   0.0262174], [0.96, 0.0294117], [0.965, 0.0337721], [0.97,   0.0399801], [0.975, 0.0493457], [0.98, 0.064705], [0.985,   0.0933618], [0.99, 0.160302], [0.995, 0.417398], [1.,   6.4085], [1.005, 4.2057], [1.01, 3.18664], [1.015, 2.75299], [1.02,   2.51029], [1.025, 2.35558], [1.03, 2.24917], [1.035,   2.17227], [1.04, 2.11475], [1.045, 2.07062], [1.05,   2.03611], [1.055, 2.00874], [1.06, 1.9868], [1.065, 1.96904], [1.07,   1.9546], [1.075, 1.94279], [1.08, 1.93311], [1.085,   1.92517], [1.09, 1.91866], [1.095, 1.91334], [1.1, 1.909], [1.105,   1.9055], [1.11, 1.9027], [1.115, 1.90048], [1.12, 1.89878], [1.125,   1.8975], [1.13, 1.89658], [1.135, 1.89598], [1.14, 1.89565], [1.145,   1.89555], [1.15, 1.89564], [1.155, 1.89591], [1.16,   1.89632], [1.165, 1.89686], [1.17, 1.8975], [1.175, 1.89824], [1.18,   1.89905], [1.185, 1.89993], [1.19, 1.90087], [1.195,   1.90185], [1.2, 1.90287], [1.205, 1.90392], [1.21, 1.90501], [1.215,   1.90611], [1.22, 1.90722], [1.225, 1.90835], [1.23,   1.90949], [1.235, 1.91063], [1.24, 1.91178], [1.245,   1.91292], [1.25, 1.91407], [1.255, 1.91521], [1.26,   1.91634], [1.265, 1.91747], [1.27, 1.91859], [1.275, 1.9197], [1.28,   1.9208], [1.285, 1.92189], [1.29, 1.92297], [1.295, 1.92403], [1.3,   1.92509], [1.305, 1.92613], [1.31, 1.92716], [1.315,   1.92817], [1.32, 1.92917], [1.325, 1.93015], [1.33,   1.93113], [1.335, 1.93208], [1.34, 1.93303], [1.345,   1.93396], [1.35, 1.93487], [1.355, 1.93577], [1.36,   1.93666], [1.365, 1.93753], [1.37, 1.93839], [1.375,   1.93924], [1.38, 1.94007], [1.385, 1.94089], [1.39,   1.94169], [1.395, 1.94248], [1.4, 1.94326], [1.405, 1.94403], [1.41,   1.94478], [1.415, 1.94552], [1.42, 1.94625], [1.425,   1.94697], [1.43, 1.94767], [1.435, 1.94836], [1.44,   1.94904], [1.445, 1.94971], [1.45, 1.95037], [1.455,   1.95102], [1.46, 1.95166], [1.465, 1.95228], [1.47, 1.9529], [1.475,   1.9535], [1.48, 1.9541], [1.485, 1.95468], [1.49, 1.95526], [1.495,   1.95583], [1.5, 1.95638]]
C=[[0., 246.595], [0.01, 298.949], [0.02, 293.513], [0.03, 287.91], [0.04, 282.15], [0.05, 276.243], [0.06, 270.203], [0.07, 264.043], [0.08, 257.78], [0.09, 251.433], [0.1, 245.019], [0.11, 238.558], [0.12, 232.072], [0.13, 225.582], [0.14, 219.109], [0.15, 212.674], [0.16, 206.299], [0.17, 200.006], [0.18, 193.814], [0.19, 187.743], [0.2, 181.813], [0.21, 176.04], [0.22, 170.442], [0.23, 165.032], [0.24, 159.826], [0.25, 154.834], [0.26, 150.067], [0.27, 145.534], [0.28, 141.241], [0.29, 137.192], [0.3, 133.391], [0.31, 129.84], [0.32, 126.536], [0.33, 123.479], [0.34, 120.663], [0.35, 118.083], [0.36, 115.732], [0.37, 113.601], [0.38, 111.68], [0.39, 109.958], [0.4, 108.422], [0.41, 107.059], [0.42, 105.854], [0.43, 104.793], [0.44, 103.861], [0.45, 103.041], [0.46, 102.319], [0.47, 101.677], [0.48, 101.1], [0.49, 100.572], [0.5, 100.078], [0.51, 99.6024], [0.52, 99.1313], [0.53, 98.6511], [0.54, 98.1487], [0.55, 97.6124], [0.56, 97.031], [0.57, 96.3948], [0.58, 95.6948], [0.59, 94.9234], [0.6, 94.0742], [0.61, 93.1418], [0.62, 92.1223], [0.63, 91.0129], [0.64, 89.8119], [0.65, 88.5188], [0.66, 87.1346], [0.67, 85.661], [0.68, 84.1009], [0.69, 82.4582], [0.7, 80.7378], [0.71, 78.9454], [0.72, 77.0874], [0.73, 75.1711], [0.74, 73.2041], [0.75, 71.1948], [0.76, 69.1518], [0.77, 67.084], [0.78, 65.0008], [0.79, 62.9115], [0.8, 60.8253], [0.81, 58.7515], [0.82, 56.6994], [0.83, 54.6777], [0.84, 52.695], [0.85, 50.7594], [0.86, 48.8787], [0.87, 47.06], [0.88, 45.3098], [0.89, 43.6341], [0.9, 42.0381], [0.91, 40.5263], [0.92, 39.1026], [0.93, 37.7698], [0.94, 36.5303], [0.95, 35.3854], [0.96, 34.336], [0.97, 33.3819], [0.98, 32.5223], [0.99, 31.7556], [1., 31.0796], [1.01, 30.4915], [1.02, 29.9876], [1.03, 29.564], [1.04, 29.216], [1.05, 28.9386], [1.06, 28.7264], [1.07, 28.5735], [1.08, 28.4738], [1.09, 28.4211], [1.1, 28.4088], [1.11, 28.4305], [1.12, 28.4794], [1.13, 28.5491], [1.14, 28.633], [1.15, 28.7247], [1.16, 28.8182], [1.17, 28.9075], [1.18, 28.987], [1.19, 29.0514], [1.2, 29.0958], [1.21, 29.1156], [1.22, 29.1069], [1.23, 29.0661], [1.24, 28.9899], [1.25, 28.8758], [1.26, 28.7218], [1.27, 28.5261], [1.28, 28.2877], [1.29, 28.006], [1.3, 27.681], [1.31, 27.313], [1.32, 26.9029], [1.33, 26.4521], [1.34, 25.9622], [1.35, 25.4355], [1.36, 24.8742], [1.37, 24.2814], [1.38, 23.66], [1.39, 23.0135], [1.4, 22.3453], [1.41, 21.6592], [1.42, 20.959], [1.43, 20.2488], [1.44, 19.5325], [1.45, 18.814], [1.46, 18.0974], [1.47, 17.3866], [1.48, 16.6854], [1.49, 15.9975], [1.5, 15.3263], [1.51, 14.6751], [1.52, 14.047], [1.53, 13.4449], [1.54, 12.8713], [1.55, 12.3284], [1.56, 11.8182], [1.57, 11.3424], [1.58, 10.9022], [1.59, 10.4987], [1.6, 10.1326], [1.61, 9.80418], [1.62, 9.51345], [1.63, 9.26013], [1.64, 9.04362], [1.65, 8.86303], [1.66, 8.71718], [1.67, 8.60466], [1.68, 8.52379], [1.69, 8.47268], [1.7, 8.44926], [1.71, 8.45127], [1.72, 8.4763], [1.73, 8.52185], [1.74, 8.5853], [1.75, 8.66395], [1.76, 8.7551], [1.77, 8.85598], [1.78, 8.96388], [1.79, 9.0761], [1.8, 9.18999], [1.81, 9.30301], [1.82, 9.41269], [1.83, 9.5167], [1.84, 9.61286], [1.85, 9.69912], [1.86, 9.77362], [1.87, 9.83468], [1.88, 9.88081], [1.89, 9.91074], [1.9, 9.92338], [1.91, 9.91787], [1.92, 9.89358], [1.93, 9.85005], [1.94, 9.78709], [1.95, 9.70467], [1.96, 9.60299], [1.97, 9.48244], [1.98, 9.3436], [1.99, 9.18722], [2., 9.01421], [2.01, 8.82564], [2.02, 8.62271], [2.03, 8.40675], [2.04, 8.17918], [2.05, 7.94152], [2.06, 7.69536], [2.07, 7.44233], [2.08, 7.18413], [2.09, 6.92245], [2.1, 6.65899], [2.11, 6.39544], [2.12, 6.13348], [2.13, 5.87471], [2.14, 5.62069], [2.15, 5.37291], [2.16, 5.13279], [2.17, 4.90161], [2.18, 4.6806], [2.19, 4.47084], [2.2, 4.2733], [2.21, 4.08882], [2.22, 3.9181], [2.23, 3.76173], [2.24, 3.62013], [2.25, 3.49359], [2.26, 3.38229], [2.27, 3.28624], [2.28, 3.20532], [2.29, 3.13931], [2.3, 3.08783], [2.31, 3.05041], [2.32, 3.02646], [2.33, 3.01529], [2.34, 3.01612], [2.35, 3.02808], [2.36, 3.05025], [2.37, 3.0816], [2.38, 3.12111], [2.39, 3.16768], [2.4, 3.22019], [2.41, 3.2775], [2.42, 3.33847], [2.43, 3.40195], [2.44, 3.46684], [2.45, 3.53201], [2.46, 3.59641], [2.47, 3.65901], [2.48, 3.71883], [2.49, 3.77497], [2.5, 3.82658], [2.51, 3.87287], [2.52, 3.91314], [2.53, 3.94679], [2.54, 3.97327], [2.55, 3.99214], [2.56, 4.00305], [2.57, 4.00572], [2.58, 3.99999], [2.59, 3.98577], [2.6, 3.96306], [2.61, 3.93194], [2.62, 3.89259], [2.63, 3.84526], [2.64, 3.79026], [2.65, 3.728], [2.66, 3.65891], [2.67, 3.58352], [2.68, 3.50239], [2.69, 3.41612], [2.7, 3.32535], [2.71, 3.23076], [2.72, 3.13304], [2.73, 3.03292], [2.74, 2.9311], [2.75, 2.8283], [2.76, 2.72526], [2.77, 2.62266], [2.78, 2.5212], [2.79, 2.42153], [2.8, 2.32429], [2.81, 2.23007], [2.82, 2.13943], [2.83, 2.05288], [2.84, 1.97087], [2.85, 1.89384], [2.86, 1.82212], [2.87, 1.75602], [2.88, 1.69579], [2.89, 1.64162], [2.9, 1.59363], [2.91, 1.5519], [2.92, 1.51643], [2.93, 1.48718], [2.94, 1.46406], [2.95, 1.44691], [2.96, 1.43555], [2.97, 1.42972], [2.98, 1.42914], [2.99, 1.43348], [3., 1.44238], [3.01, 1.45545], [3.02, 1.47228], [3.03, 1.49241], [3.04, 1.51541], [3.05, 1.5408], [3.06, 1.5681], [3.07, 1.59684], [3.08, 1.62653], [3.09, 1.65671], [3.1, 1.68692], [3.11, 1.7167], [3.12, 1.74564], [3.13, 1.77331], [3.14, 1.79935], [3.15, 1.82339], [3.16, 1.84511], [3.17, 1.86422], [3.18, 1.88045], [3.19, 1.89359], [3.2, 1.90345], [3.21, 1.90988], [3.22, 1.91277], [3.23, 1.91206], [3.24, 1.90769], [3.25, 1.89969], [3.26, 1.88808], [3.27, 1.87293], [3.28, 1.85436], [3.29, 1.83251], [3.3, 1.80752], [3.31, 1.77961], [3.32, 1.74899], [3.33, 1.71588], [3.34, 1.68056], [3.35, 1.64329], [3.36, 1.60435], [3.37, 1.56405], [3.38, 1.52267], [3.39, 1.48053], [3.4, 1.43794], [3.41, 1.39518], [3.42, 1.35257], [3.43, 1.31039], [3.44, 1.26891], [3.45, 1.22842], [3.46, 1.18915], [3.47, 1.15134], [3.48, 1.11521], [3.49, 1.08096], [3.5, 1.04876], [3.51, 1.01876], [3.52, 0.991085], [3.53, 0.965846], [3.54, 0.94312], [3.55, 0.922962], [3.56, 0.905403], [3.57, 0.890449], [3.58, 0.878083], [3.59, 0.868265], [3.6, 0.860934], [3.61, 0.856006], [3.62, 0.85338], [3.63, 0.852937], [3.64, 0.854539], [3.65, 0.858036], [3.66, 0.863264], [3.67, 0.870049], [3.68, 0.878205], [3.69, 0.887543], [3.7, 0.897865], [3.71, 0.908972], [3.72, 0.920664], [3.73, 0.93274], [3.74, 0.945002], [3.75, 0.957257], [3.76, 0.969317], [3.77, 0.981003], [3.78, 0.992142], [3.79, 1.00258], [3.8, 1.01215], [3.81, 1.02074], [3.82, 1.02821], [3.83, 1.03446], [3.84, 1.03939], [3.85, 1.04293], [3.86, 1.04502], [3.87, 1.0456], [3.88, 1.04465], [3.89, 1.04215], [3.9, 1.03811], [3.91, 1.03254], [3.92, 1.02547], [3.93, 1.01695], [3.94, 1.00704], [3.95, 0.995797], [3.96, 0.983314], [3.97, 0.969679], [3.98, 0.954992], [3.99, 0.939362], [4., 0.922903], [4.01, 0.905735], [4.02, 0.887983], [4.03, 0.869772], [4.04, 0.851232], [4.05, 0.832492], [4.06, 0.813678], [4.07, 0.794916], [4.08, 0.776329], [4.09, 0.758034], [4.1, 0.740144], [4.11, 0.722764], [4.12, 0.705995], [4.13, 0.689926], [4.14, 0.674642], [4.15, 0.660216], [4.16, 0.646711], [4.17, 0.634182], [4.18, 0.622673], [4.19, 0.612218], [4.2, 0.60284], [4.21, 0.594553], [4.22, 0.587359], [4.23, 0.581252], [4.24, 0.576214], [4.25, 0.57222], [4.26, 0.569236], [4.27, 0.567219], [4.28, 0.566119], [4.29, 0.565878], [4.3, 0.566433], [4.31, 0.567716], [4.32, 0.569653], [4.33, 0.572167], [4.34, 0.575177], [4.35, 0.578602], [4.36, 0.582356], [4.37, 0.586357], [4.38, 0.590519], [4.39, 0.59476], [4.4, 0.598999], [4.41, 0.603156], [4.42, 0.607155], [4.43, 0.610926], [4.44, 0.614401], [4.45, 0.617516], [4.46, 0.620214], [4.47, 0.622445], [4.48, 0.624162], [4.49, 0.625327], [4.5, 0.625906], [4.51, 0.625874], [4.52, 0.62521], [4.53, 0.623904], [4.54, 0.621948], [4.55, 0.619344], [4.56, 0.616098], [4.57, 0.612223], [4.58, 0.607738], [4.59, 0.602668], [4.6, 0.597042], [4.61, 0.590895], [4.62, 0.584264], [4.63, 0.577192], [4.64, 0.569725], [4.65, 0.561911], [4.66, 0.553801], [4.67, 0.545446], [4.68, 0.536901], [4.69, 0.528219], [4.7, 0.519454], [4.71, 0.510661], [4.72, 0.501891], [4.73, 0.493198], [4.74, 0.484629], [4.75, 0.476233], [4.76, 0.468054], [4.77, 0.460134], [4.78, 0.452512], [4.79, 0.445222], [4.8, 0.438296], [4.81, 0.431759], [4.82, 0.425637], [4.83, 0.419946], [4.84, 0.414702], [4.85, 0.409913], [4.86, 0.405586], [4.87, 0.401723], [4.88, 0.398319], [4.89, 0.395369], [4.9, 0.392861], [4.91, 0.390781], [4.92, 0.389111], [4.93, 0.38783], [4.94, 0.386914], [4.95, 0.386336], [4.96, 0.386068], [4.97, 0.386078], [4.98, 0.386335], [4.99, 0.386804], [5., 0.38745]]
D=[[0., 128.004], [0.01, 160.303], [0.02, 162.179], [0.03, 165.555], [0.04, 170.337], [0.05, 176.419], [0.06, 183.68], [0.07, 191.99], [0.08, 201.208], [0.09, 211.187], [0.1, 221.773], [0.11, 232.809], [0.12, 244.137], [0.13, 255.596], [0.14, 267.03], [0.15, 278.284], [0.16, 289.21], [0.17, 299.666], [0.18, 309.519], [0.19, 318.644], [0.2, 326.929], [0.21, 334.272], [0.22, 340.586], [0.23, 345.796], [0.24, 349.843], [0.25, 352.68], [0.26, 354.278], [0.27, 354.621], [0.28, 353.708], [0.29, 351.554], [0.3, 348.187], [0.31, 343.648], [0.32, 337.993], [0.33, 331.288], [0.34, 323.61], [0.35, 315.048], [0.36, 305.696], [0.37, 295.659], [0.38, 285.046], [0.39, 273.971], [0.4, 262.553], [0.41, 250.91], [0.42, 239.162], [0.43, 227.428], [0.44, 215.826], [0.45, 204.468], [0.46, 193.463], [0.47, 182.913], [0.48, 172.913], [0.49, 163.551], [0.5, 154.906], [0.51, 147.046], [0.52, 140.031], [0.53, 133.909], [0.54, 128.717], [0.55, 124.48], [0.56, 121.214], [0.57, 118.922], [0.58, 117.597], [0.59, 117.218], [0.6, 117.759], [0.61, 119.18], [0.62, 121.433], [0.63, 124.461], [0.64, 128.2], [0.65, 132.58], [0.66, 137.524], [0.67, 142.949], [0.68, 148.771], [0.69, 154.9], [0.7, 161.247], [0.71, 167.721], [0.72, 174.231], [0.73, 180.689], [0.74, 187.008], [0.75, 193.105], [0.76, 198.9], [0.77, 204.321], [0.78, 209.298], [0.79, 213.771], [0.8, 217.684], [0.81, 220.991], [0.82, 223.653], [0.83, 225.64], [0.84, 226.927], [0.85, 227.503], [0.86, 227.362], [0.87, 226.507], [0.88, 224.949], [0.89, 222.707], [0.9, 219.81], [0.91, 216.29], [0.92, 212.188], [0.93, 207.552], [0.94, 202.432], [0.95, 196.886], [0.96, 190.974], [0.97, 184.759], [0.98, 178.309], [0.99, 171.69], [1., 164.97], [1.01, 158.218], [1.02, 151.501], [1.03, 144.886], [1.04, 138.435], [1.05, 132.208], [1.06, 126.264], [1.07, 120.654], [1.08, 115.425], [1.09, 110.622], [1.1, 106.28], [1.11, 102.431], [1.12, 99.1], [1.13, 96.3051], [1.14, 94.0585], [1.15, 92.366], [1.16, 91.227], [1.17, 90.6345], [1.18, 90.5753], [1.19, 91.0309], [1.2, 91.9767], [1.21, 93.3835], [1.22, 95.2172], [1.23, 97.4395], [1.24, 100.009], [1.25, 102.88], [1.26, 106.005], [1.27, 109.335], [1.28, 112.818], [1.29, 116.404], [1.3, 120.041], [1.31, 123.677], [1.32, 127.263], [1.33, 130.749], [1.34, 134.089], [1.35, 137.239], [1.36, 140.159], [1.37, 142.812], [1.38, 145.163], [1.39, 147.183], [1.4, 148.849], [1.41, 150.138], [1.42, 151.037], [1.43, 151.533], [1.44, 151.622], [1.45, 151.302], [1.46, 150.578], [1.47, 149.456], [1.48, 147.951], [1.49, 146.08], [1.5, 143.863], [1.51, 141.325], [1.52, 138.493], [1.53, 135.4], [1.54, 132.077], [1.55, 128.561], [1.56, 124.888], [1.57, 121.095], [1.58, 117.223], [1.59, 113.31], [1.6, 109.394], [1.61, 105.514], [1.62, 101.706], [1.63, 98.0067], [1.64, 94.449], [1.65, 91.0646], [1.66, 87.8822], [1.67, 84.9279], [1.68, 82.2247], [1.69, 79.7924], [1.7, 77.6472], [1.71, 75.8017], [1.72, 74.2652], [1.73, 73.043], [1.74, 72.1368], [1.75, 71.5447], [1.76, 71.2615], [1.77, 71.2781], [1.78, 71.5827], [1.79, 72.1601], [1.8, 72.9926], [1.81, 74.0595], [1.82, 75.3384], [1.83, 76.8045], [1.84, 78.4315], [1.85, 80.1919], [1.86, 82.0572], [1.87, 83.9981], [1.88, 85.9851], [1.89, 87.989], [1.9, 89.9807], [1.91, 91.932], [1.92, 93.8156], [1.93, 95.6057], [1.94, 97.2779], [1.95, 98.8099], [1.96, 100.181], [1.97, 101.374], [1.98, 102.372], [1.99, 103.163], [2., 103.736], [2.01, 104.083], [2.02, 104.2], [2.03, 104.083], [2.04, 103.735], [2.05, 103.158], [2.06, 102.357], [2.07, 101.342], [2.08, 100.123], [2.09, 98.7122], [2.1, 97.1253], [2.11, 95.3787], [2.12, 93.4903], [2.13, 91.4798], [2.14, 89.3675], [2.15, 87.175], [2.16, 84.9239], [2.17, 82.6365], [2.18, 80.335], [2.19, 78.0412], [2.2, 75.7767], [2.21, 73.5623], [2.22, 71.4177], [2.23, 69.3617], [2.24, 67.4114], [2.25, 65.5829], [2.26, 63.8901], [2.27, 62.3454], [2.28, 60.9593], [2.29, 59.7403], [2.3, 58.6946], [2.31, 57.8267], [2.32, 57.1386], [2.33, 56.6307], [2.34, 56.3009], [2.35, 56.1453], [2.36, 56.1583], [2.37, 56.3321], [2.38, 56.6576], [2.39, 57.1238], [2.4, 57.7188], [2.41, 58.4289], [2.42, 59.2399], [2.43, 60.1363], [2.44, 61.1022], [2.45, 62.1212], [2.46, 63.1766], [2.47, 64.2515], [2.48, 65.3293], [2.49, 66.3937], [2.5, 67.4287], [2.51, 68.4191], [2.52, 69.3506], [2.53, 70.2097], [2.54, 70.9841], [2.55, 71.6628], [2.56, 72.236], [2.57, 72.6953], [2.58, 73.034], [2.59, 73.2466], [2.6, 73.3295], [2.61, 73.2804], [2.62, 73.0989], [2.63, 72.7859], [2.64, 72.3438], [2.65, 71.7766], [2.66, 71.0895], [2.67, 70.2893], [2.68, 69.3837], [2.69, 68.3815], [2.7, 67.2927], [2.71, 66.1278], [2.72, 64.8983], [2.73, 63.616], [2.74, 62.2933], [2.75, 60.9427], [2.76, 59.5767], [2.77, 58.2082], [2.78, 56.8493], [2.79, 55.5123], [2.8, 54.2088], [2.81, 52.9497], [2.82, 51.7455], [2.83, 50.6056], [2.84, 49.5387], [2.85, 48.5524], [2.86, 47.6533], [2.87, 46.8469], [2.88, 46.1376], [2.89, 45.5283], [2.9, 45.0213], [2.91, 44.6171], [2.92, 44.3154], [2.93, 44.1147], [2.94, 44.0122], [2.95, 44.0043], [2.96, 44.0861], [2.97, 44.2521], [2.98, 44.4958], [2.99, 44.8099], [3., 45.1866], [3.01, 45.6173], [3.02, 46.0932], [3.03, 46.605], [3.04, 47.1434], [3.05, 47.6987], [3.06, 48.2614], [3.07, 48.8221], [3.08, 49.3715], [3.09, 49.9008], [3.1, 50.4014], [3.11, 50.8654], [3.12, 51.2854], [3.13, 51.6546], [3.14, 51.9671], [3.15, 52.2175], [3.16, 52.4016], [3.17, 52.5157], [3.18, 52.5572], [3.19, 52.5244], [3.2, 52.4164], [3.21, 52.2332], [3.22, 51.9757], [3.23, 51.6457], [3.24, 51.2457], [3.25, 50.7792], [3.26, 50.2501], [3.27, 49.6631], [3.28, 49.0236], [3.29, 48.3374], [3.3, 47.6106], [3.31, 46.85], [3.32, 46.0624], [3.33, 45.2549], [3.34, 44.4346], [3.35, 43.6088], [3.36, 42.7846], [3.37, 41.969], [3.38, 41.1689], [3.39, 40.3906], [3.4, 39.6403], [3.41, 38.9238], [3.42, 38.2463], [3.43, 37.6123], [3.44, 37.0262], [3.45, 36.4912], [3.46, 36.0104], [3.47, 35.5859], [3.48, 35.2191], [3.49, 34.9111], [3.5, 34.6618], [3.51, 34.4708], [3.52, 34.337], [3.53, 34.2585], [3.54, 34.233], [3.55, 34.2576], [3.56, 34.3289], [3.57, 34.4429], [3.58, 34.5954], [3.59, 34.7818], [3.6, 34.9971], [3.61, 35.2362], [3.62, 35.4937], [3.63, 35.7642], [3.64, 36.0424], [3.65, 36.3227], [3.66, 36.5998], [3.67, 36.8685], [3.68, 37.1239], [3.69, 37.3612], [3.7, 37.5761], [3.71, 37.7644], [3.72, 37.9224], [3.73, 38.0471], [3.74, 38.1354], [3.75, 38.1852], [3.76, 38.1946], [3.77, 38.1624], [3.78, 38.0877], [3.79, 37.9702], [3.8, 37.8103], [3.81, 37.6085], [3.82, 37.3661], [3.83, 37.0848], [3.84, 36.7665], [3.85, 36.4139], [3.86, 36.0297], [3.87, 35.617], [3.88, 35.1793], [3.89, 34.7203], [3.9, 34.2436], [3.91, 33.7535], [3.92, 33.2538], [3.93, 32.7487], [3.94, 32.2423], [3.95, 31.7387], [3.96, 31.2417], [3.97, 30.7553], [3.98, 30.2829], [3.99, 29.828], [4., 29.3938]]
x=[]
x2=[]
y=[]
y2=[]
for i in :
    x.append(i[0])
    y.append(i[1])
#for i in A:
#    x2.append(i[0])
#    y2.append(i[1])

x=np.array(x)
y=np.array(y)

#plt.plot(x,170*np.exp(-1.5*x)*(1+0.8*np.cos(4.4*x)**2))
plt.plot(x,y)
#plt.plot(x2,y2)

In [313]:
def f(x,a,b,xi,xi2,k):
    return a*np.exp(-xi*x)+b*np.cos(k*x)**2*np.exp(-xi2*x)

temp=sp.optimize.curve_fit(f,x,y,p0=[400,1.0,1.2,1.2,5.5])[0]
plt.plot(x,y)
plt.plot(x,f(x,temp[0],temp[1],temp[2],temp[3],temp[4]))
temp

array([508.81621905, 265.77022745,   1.47007554,   0.91481814,
         5.43193859])